# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#load csv and create df
city_weather_data = pd.read_csv('../output_data/weather_dataframe.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#columns for heatmap, making sure they are in number format
locations = city_weather_data[["Lat", "Long"]].astype(float)
humidity =city_weather_data['Humidity'].astype(int)

fig = gmaps.figure()

#create heatlayer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
#add heatlayer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#MY ideal weather conditions
ideal_weather = city_weather_data.loc[((city_weather_data['Max Temp'] <= 80) &
                                       (city_weather_data['Humidity'] <= 35) &
                                       (city_weather_data['Wind Speed'] <= 10)&
                                       (city_weather_data['Cloudiness'] <= 40)),:]
ideal_weather

,Unnamed: 0,City,Country,Lat,Long,Date,Max Temp,Humidity,Cloudiness,Wind Speed
115,115,Svarstad,NO,59.40,9.97,1592425926,64.99,1,0,1.23
122,122,Srednekolymsk,RU,67.45,153.68,1592425793,78.12,34,0,7.81
379,379,Dawlatabad,AF,36.41,64.91,1592425975,77.49,17,0,4.59
384,384,Chitral,PK,35.85,71.79,1592425976,69.87,25,0,6.64
424,424,Miraflores,MX,19.22,-98.81,1592425983,78.80,27,40,8.05
523,523,West Wendover,US,40.74,-114.07,1592425999,64.40,23,1,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Just using this cell to call an individual response to look at data
#base api url
places_api = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
lat = 37.19	
long = 10.18
search_radius=5000
place_type='lodging'
keyword='hotel'
query_url=f'{places_api}location={lat},{long}&radius={search_radius}&type={place_type}&keyword={keyword}&key={g_key}'
testrun = requests.get(f'{query_url}').json()
#pprint(testrun)
#testrun['results'][0]#['name']

In [7]:
#base url with parameters for data I need
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key}

#new list to append when getting responses
hotel_names=[]


for index, row in ideal_weather.iterrows():
    latitudes = row["Lat"]
    longitudes = row["Long"]
    
    #add locations paramter from ideal_weather df
    params["location"] = f"{latitudes},{longitudes}"

    # make request and print process log
    response = requests.get(base_url, params=params)
    hotel_query= response.json()
    print(f"Processing request for {row['City']}")
     
    #some cities will not have a nearby hotel in the given parameters, use try statement      
    try:
        hotel_names.append(hotel_query['results'][0]['name'])
        print(f"Nearest hotel: {hotel_query['results'][0]['name']}")
    except (KeyError, IndexError):
        hotel_names.append('N/A')
        print("Missing field/result... skipping.")          
    print('----------------------------')
 

Processing request for Svarstad
Nearest hotel: Sogn Gjestegard
----------------------------
Processing request for Srednekolymsk
Missing field/result... skipping.
----------------------------
Processing request for Dawlatabad
Missing field/result... skipping.
----------------------------
Processing request for Chitral
Nearest hotel: Al Mughal Hotel
----------------------------
Processing request for Miraflores
Nearest hotel: Hotel Mansión
----------------------------
Processing request for West Wendover
Nearest hotel: Knights Inn Wendover
----------------------------


In [8]:
#create new column from the hotel names list
#had to use .copy for some reason not entirely sure why, getting an error if i didnt
hotel_df=ideal_weather.copy()
hotel_df['Hotel Name'] = hotel_names
hotel_df

,Unnamed: 0,City,Country,Lat,Long,Date,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name
115,115,Svarstad,NO,59.40,9.97,1592425926,64.99,1,0,1.23,Sogn Gjestegard
122,122,Srednekolymsk,RU,67.45,153.68,1592425793,78.12,34,0,7.81,N/A
379,379,Dawlatabad,AF,36.41,64.91,1592425975,77.49,17,0,4.59,N/A
384,384,Chitral,PK,35.85,71.79,1592425976,69.87,25,0,6.64,Al Mughal Hotel
424,424,Miraflores,MX,19.22,-98.81,1592425983,78.80,27,40,8.05,Hotel Mansión
523,523,West Wendover,US,40.74,-114.07,1592425999,64.40,23,1,4.70,Knights Inn Wendover


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
# Add the layer to the map
fig.add_layer(markers)
# Display Map
fig

Figure(layout=FigureLayout(height='420px'))